In [5]:
import numpy as np 
import pandas as pd 
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt 

In [8]:
from sklearn.model_selection import train_test_split

# # Data inladen
df = pd.read_csv("final_df.csv")
print(df.head())
# df = groot_gemid_df

# Eerst de data opschudden om bias te voorkomen
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Zorg ervoor dat elke ervaring, geslacht en gewichtsklasse in elke set vertegenwoordigd zijn
train_data = pd.DataFrame()
val_data = pd.DataFrame()
test_data = pd.DataFrame()

for ervaring in df['ervaring'].unique():
    for geslacht in df['binary_geslacht'].unique():
        for gewichtsklasse in df['binary_gewichtsklasse'].unique():
            subset = df[(df['ervaring'] == ervaring) & (df['binary_geslacht'] == geslacht) & (df['binary_gewichtsklasse'] == gewichtsklasse)]
            if not subset.empty:
                temp_train, temp_temp = train_test_split(subset, test_size=0.3, random_state=42)
                temp_val, temp_test = train_test_split(temp_temp, test_size=0.3, random_state=42)
                train_data = pd.concat([train_data, temp_train])
                val_data = pd.concat([val_data, temp_val])
                test_data = pd.concat([test_data, temp_test])

# Reset indexen
train_data.reset_index(drop=True, inplace=True)
val_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

# Controleren op juiste verdeling
print(f"Trainingsdata: {len(train_data)} rijen")
print(f"Validatiedata: {len(val_data)} rijen")
print(f"Testdata: {len(test_data)} rijen")

# Optioneel: data opslaan in aparte bestanden
train_data.to_csv('train_data.csv', index=False)
val_data.to_csv('val_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)

   ervaring trainingtype  500_split  2k tijd  binary_trainingtype  \
0         1         5x5'      104.6    379.9                    0   
1         1         5x5'      104.7    379.9                    0   
2         1         5x5'      104.3    379.9                    0   
3         1         5x5'      104.0    379.9                    0   
4         1         5x5'      104.1    379.9                    0   

   binary_geslacht  binary_gewichtsklasse  
0                0                      1  
1                0                      1  
2                0                      1  
3                0                      1  
4                0                      1  
Trainingsdata: 3018 rijen
Validatiedata: 905 rijen
Testdata: 391 rijen


In [9]:
X_train = train_data.drop(columns=['2k tijd'])
y_train = train_data['2k tijd']

X_val = val_data.drop(columns=['2k tijd'])
y_val = val_data['2k tijd']

X_test = test_data.drop(columns=['2k tijd'])
y_test = test_data['2k tijd']

# Ensure all datasets have the same columns after encoding
X_train, X_test = X_train.align(X_val, join='outer', axis=1, fill_value=0)
X_train, X_test = X_train.align(X_test, join='outer', axis=1, fill_value=0)

# Step 3: Verify data types
print(X_train.dtypes)

ridge_model = Ridge()
ridge_model.fit(X_train, y_train)

val_predictions = ridge_model.predict(X_val)
val_mse = mean_squared_error(y_val, val_predictions)
print(f"Validation MSE: {val_mse}")

# Optionally evaluate on test data
test_predictions = ridge_model.predict(X_test)
test_mse = mean_squared_error(y_test, test_predictions)
print(f"Test MSE: {test_mse}")

ervaring                   int64
trainingtype              object
500_split                float64
binary_trainingtype        int64
binary_geslacht            int64
binary_gewichtsklasse      int64
dtype: object


ValueError: could not convert string to float: "3x7'"